In [4]:
import re
import pandas as pd
import numpy as np

## Plotting Libraries
import seaborn as sns
import matplotlib.pyplot as plt

## Pytorch Imports
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim

## NLP Libraries
import spacy
from sklearn.model_selection import train_test_split
from nltk import download
import gensim
from nltk.corpus import stopwords
spacy_en = spacy.load('en')
download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
## Checking if GPU is available
torch.cuda.is_available()

True

In [6]:
a = torch.cuda.FloatTensor([1])
print("{} - {}".format(type(a),type(a[0])))

<class 'torch.cuda.FloatTensor'> - <class 'float'>


## 1. Reading Dataset

In [7]:
train = pd.read_csv('train.csv')
print(len(train))
train.tail()

19579


,id,text,author
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL


In [8]:
test = pd.read_csv('test.csv')
print(len(test))
test.tail()

8392


,id,text
8387,id11749,All this is now the fitter for my purpose.
8388,id10526,I fixed myself on a wide solitude.
8389,id13477,It is easily understood that what might improv...
8390,id13761,"Be this as it may, I now began to feel the ins..."
8391,id04282,"Long winded, statistical, and drearily genealo..."


In [9]:
## Checking dataset unbalance

In [10]:
EAP = train[train['author']=='EAP'].reset_index()
EAP_size = len(EAP)
print(EAP_size)
EAP.tail()

7900


,index,id,text,author
7895,19572,id03325,But these and other difficulties attending res...,EAP
7896,19574,id17718,"I could have fancied, while I looked at it, th...",EAP
7897,19575,id08973,The lids clenched themselves together as if in...,EAP
7898,19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
7899,19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [11]:
HPL = train[train['author']=='HPL'].reset_index()
HPL_size = len(HPL)
print(HPL_size)
HPL.tail()

5635


,index,id,text,author
5630,19554,id07976,"They admitted they had been drunk, but both vo...",HPL
5631,19559,id18823,When a fumbling came in the nearer casements h...,HPL
5632,19561,id08678,Average people in society and business New Eng...,HPL
5633,19571,id14420,"My watch was still going, and told me that the...",HPL
5634,19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL


In [12]:
MWS = train[train['author']=='MWS'].reset_index()
MWS_size = len(MWS)
print(MWS_size)
MWS.tail()

6044


,index,id,text,author
6039,19563,id10563,Yet from whom has not that rude hand rent away...,MWS
6040,19566,id00832,"These reflections made our legislators pause, ...",MWS
6041,19569,id26790,Once my fancy was soothed with dreams of virtu...,MWS
6042,19570,id14263,"Nay, you may have met with another whom you ma...",MWS
6043,19573,id07567,Stress of weather drove us up the Adriatic Gul...,MWS


In [13]:
EAP[0:HPL_size].tail()

,index,id,text,author
5630,13951,id16972,I saw them now even more unequivocally than I ...,EAP
5631,13952,id05477,For some time his countenance had been losing ...,EAP
5632,13954,id17337,"""It is done it is most cheerfully agreed.",EAP
5633,13956,id05256,"""At this stage of my reflections I endeavored ...",EAP
5634,13959,id12599,Coincidences ten times as remarkable as this t...,EAP


In [14]:
train_undersampled = pd.concat([EAP[0:HPL_size],HPL,MWS[0:HPL_size]], ignore_index=True)
train_undersampled.drop(['index'],axis=1,inplace=True)
train_undersampled = train_undersampled.sample(frac=1).reset_index(drop=True)
train_undersampled.tail()

,id,text,author
16900,id11176,Muffled sounds of possible vocal origin approa...,HPL
16901,id25516,It was that of an ancient Puritan interior a h...,HPL
16902,id21478,"At evening Iranon sang, and while he sang an o...",HPL
16903,id17472,"As before, he dreamed first of the village tha...",HPL
16904,id00605,So matters went till that night when Williams ...,HPL


In [15]:
(len(train))

19579

## 3. Preprocessing

In [16]:
def transformText(text, do_stop=False, do_stem=False):
    
    stops = set(stopwords.words("english"))
    
    # Convert text to lower
    text = text.lower()
    
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Removing all the stopwords
    
    if (do_stop==True):
        filtered_words = [word for word in text.split() if word not in stops]
    else:
        filtered_words = [word for word in text.split()]

    # Removing all the tokens with lesser than 3 characters
    filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=2)
    
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    
    # Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    if (do_stem==True):
        # Stemming
        text = gensim.parsing.preprocessing.stem_text(text)
    return text

In [23]:
train['phrase_preprocessed']=train['text'].apply(lambda x: transformText(x,do_stop=False, do_stem=False))
train.tail()

,id,text,author,label,phrase_preprocessed
19574,id17718,"I could have fancied, while I looked at it, th...",EAP,0,could have fancied while looked at it that som...
19575,id08973,The lids clenched themselves together as if in...,EAP,0,the lids clenched themselves together as if in...
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP,0,mais il faut agir that is to say frenchman nev...
19577,id17513,"For an item of news like this, it strikes us i...",EAP,0,for an item of news like this it strikes us it...
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL,1,he laid gnarled claw on my shoulder and it see...


In [24]:
train_undersampled['phrase_preprocessed']=train_undersampled['text'].apply(lambda x: transformText(x,do_stop=False, do_stem=False))
train_undersampled.tail()

,id,text,author,phrase_preprocessed
16900,id11176,Muffled sounds of possible vocal origin approa...,HPL,muffled sounds of possible vocal origin approa...
16901,id25516,It was that of an ancient Puritan interior a h...,HPL,it was that of an ancient puritan interior hea...
16902,id21478,"At evening Iranon sang, and while he sang an o...",HPL,at evening iranon sang and while he sang an ol...
16903,id17472,"As before, he dreamed first of the village tha...",HPL,as before he dreamed first of the village that...
16904,id00605,So matters went till that night when Williams ...,HPL,so matters went till that night when williams ...


In [18]:
label_to_ix = { "EAP": 0, "HPL": 1, "MWS": 2 }
train['label']=[label_to_ix[a] for a in train.author]
train[0:10]

,id,text,author,label
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1
5,id22965,"A youth passed in solitude, my best years spen...",MWS,2
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP,0
7,id13515,The surcingle hung in ribands from my body.,EAP,0
8,id19322,I knew that you could not say to yourself 'ste...,EAP,0
9,id00912,I confess that neither the structure of langua...,MWS,2


In [19]:
test['phrase_preprocessed']=test['text'].apply(lambda x: transformText(x,do_stop=False, do_stem=False))
test.head()

,id,text,phrase_preprocessed
0,id02310,"Still, as I urged our leaving Ireland with suc...",still as urged our leaving ireland with such i...
1,id24541,"If a fire wanted fanning, it could readily be ...",if fire wanted fanning it could readily be fan...
2,id00134,And when they had broken down the frail door t...,and when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...,while was thinking how should possibly manage ...
4,id04081,I am not sure to what limit his knowledge may ...,am not sure to what limit his knowledge may ex...


## 4. Train/Test split, Vocab

In [25]:
x_train, x_valid, y_train, y_valid = train_test_split(train['phrase_preprocessed'],
                                                      train['author'], 
                                                      test_size=0.2)

In [26]:
x_test = np.array(test['phrase_preprocessed'])
x_test

array([ 'still as urged our leaving ireland with such inquietude and impatience my father thought it best to yield ',
       'if fire wanted fanning it could readily be fanned with newspaper and as the government grew weaker have no doubt that leather and iron acquired durability in proportion for in very short time there was not pair of bellows in all rotterdam that ever stood in need of stitch or required the assistance of hammer ',
       'and when they had broken down the frail door they found only this two cleanly picked human skeletons on the earthen floor and number of singular beetles crawling in the shadowy corners ',
       ...,
       'it is easily understood that what might improve closely scrutinized detail may at the same time injure general or more distantly observed effect ',
       'be this as it may now began to feel the inspiration of burning hope and at length nurtured in my secret thoughts stern and desperate resolution that would submit no longer to be enslaved ',

In [27]:
## Build Vocabulary
word_to_ix = {}
for sent in list(x_train) + list(x_valid) + list(x_test):
    for word in sent.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

In [28]:
print("Vocabulary size: {}".format(len(word_to_ix)))

Vocabulary size: 28307


In [29]:
label_to_ix = { "EAP": 0, "HPL": 1, "MWS": 2 }

In [30]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = len(label_to_ix)
VOCAB_SIZE, NUM_LABELS

(28307, 3)

## 5. Making dataset iterable

In [31]:
train_data=list(zip(x_train,y_train))
train_data[0:5]

[('saw lord raymond on his charger small company of officers had gathered about him and behind was promiscuous concourse of soldiers and subalterns their discipline lost their arms thrown aside no music sounded no banners streamed ',
  'MWS'),
 ('its shrill loathsome tittering stuck more and more in gilman s head and he could remember in the morning how it had pronounced the words azathoth and nyarlathotep ',
  'HPL'),
 ('the lady whom was much astonished to hear addressed as madame joyeuse after the description of madame joyeuse she had just given blushed up to the eyebrows and seemed exceedingly abashed at the reproof ',
  'EAP'),
 ('but everteeming nature will create another and another and thou wilt loose nought by my destruction ',
  'MWS'),
 ('for that the man always watched and listened no one could doubt ', 'HPL')]

In [32]:
valid_data=list(zip(x_valid,y_valid))
valid_data[0:5]

[('need not go into details ', 'EAP'),
 ('but wish to call your attention to the distinction which is made between drowned bodies and bodies thrown into the water immediately after death by violence ',
  'EAP'),
 (' do not catechise me lionel will do my duty by her be assured ', 'MWS'),
 ('have indeed no abhorrence of danger except in its absolute effect in terror ',
  'EAP'),
 ('it echoed and echoed through the dim vaultings of that ancient and nitrous cellar and had to choke back flood of reaction that threatened to burst out as hysterical laughter ',
  'HPL')]

In [33]:
def make_context_vector(seq, to_ix):
    idxs = [to_ix[w] for w in seq.split()]
    tensor = torch.cuda.LongTensor(idxs)
    return tensor

In [34]:
def make_target(label, label_to_idx):
    return torch.cuda.LongTensor([label_to_idx[label]])

## 6. Model - LSTM Classifier with Word2vec

In [35]:
!ls ../../vectors/

glove.42B.300d.txt  GoogleNews-vectors-negative300.bin


In [38]:
glove_path = '../../vectors/glove.42B.300d.txt'

In [39]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [40]:
glove_vector = loadGloveModel(glove_path)

Loading Glove Model
Done. 1917494  words loaded!


In [41]:
glove_vector['start'][0:100]

array([-0.3013  ,  0.49315 , -0.47456 , -0.55259 , -0.079275, -0.069589,
       -3.5792  ,  0.28671 , -0.034099, -0.23643 , -0.042276,  0.25939 ,
       -0.080374, -0.25137 ,  0.22347 ,  0.22123 ,  0.077188,  0.41157 ,
       -0.1154  , -0.065081, -0.1549  , -0.08466 ,  0.078873, -0.109   ,
       -0.018008,  0.2591  ,  0.09544 , -0.33333 , -0.24942 , -0.29616 ,
       -0.60454 , -0.11145 ,  0.21661 , -0.17045 ,  0.2329  ,  0.24699 ,
       -0.37996 , -0.035888, -0.035235,  0.14275 , -0.13491 ,  0.018954,
        0.35121 , -0.34182 ,  0.041529, -0.48883 ,  0.29649 ,  0.25711 ,
        0.21757 , -0.04452 ,  0.49266 ,  0.30706 , -0.30375 ,  0.12427 ,
       -0.090547,  0.055427, -0.357   ,  0.049935, -0.12842 ,  0.037939,
        0.20706 ,  0.24645 ,  0.45067 ,  0.26154 , -0.39262 ,  0.10625 ,
       -0.090539,  0.13802 ,  0.088777, -0.4219  , -0.14283 ,  0.053959,
       -0.40817 , -0.13262 , -0.44289 ,  0.046249,  0.53221 ,  0.030767,
        0.06197 , -0.18862 , -0.28016 ,  0.09877 , 

In [ ]:
#from gensim.models import KeyedVectors

In [ ]:
#w2v = KeyedVectors.load_word2vec_format('../../vectors/GoogleNews-vectors-negative300.bin', binary = True)

In [42]:
W2V_DIM = 300
## standard deviation to use
sd = 1/np.sqrt(W2V_DIM)
## Random initialization
weights = np.random.normal(0, scale=sd, size=[VOCAB_SIZE, W2V_DIM])
weights = weights.astype(np.float32)

In [43]:
for word in word_to_ix:
    id = word_to_ix.get(word,None)
    if id is not None:
        try:
            #weights[id]=w2v.wv.word_vec(word)
            weights[id]=glove_vector[word]
        except:
            weights[id]=np.random.normal(0, scale=sd, size=[1, W2V_DIM]) ## If word not present, initialize randomly

In [44]:
word_to_ix

{'saw': 0,
 'lord': 1,
 'raymond': 2,
 'on': 3,
 'his': 4,
 'charger': 5,
 'small': 6,
 'company': 7,
 'of': 8,
 'officers': 9,
 'had': 10,
 'gathered': 11,
 'about': 12,
 'him': 13,
 'and': 14,
 'behind': 15,
 'was': 16,
 'promiscuous': 17,
 'concourse': 18,
 'soldiers': 19,
 'subalterns': 20,
 'their': 21,
 'discipline': 22,
 'lost': 23,
 'arms': 24,
 'thrown': 25,
 'aside': 26,
 'no': 27,
 'music': 28,
 'sounded': 29,
 'banners': 30,
 'streamed': 31,
 'its': 32,
 'shrill': 33,
 'loathsome': 34,
 'tittering': 35,
 'stuck': 36,
 'more': 37,
 'in': 38,
 'gilman': 39,
 's': 40,
 'head': 41,
 'he': 42,
 'could': 43,
 'remember': 44,
 'the': 45,
 'morning': 46,
 'how': 47,
 'it': 48,
 'pronounced': 49,
 'words': 50,
 'azathoth': 51,
 'nyarlathotep': 52,
 'lady': 53,
 'whom': 54,
 'much': 55,
 'astonished': 56,
 'to': 57,
 'hear': 58,
 'addressed': 59,
 'as': 60,
 'madame': 61,
 'joyeuse': 62,
 'after': 63,
 'description': 64,
 'she': 65,
 'just': 66,
 'given': 67,
 'blushed': 68,
 'up': 6

In [45]:
glove_vector["confessed"][0:50]
#w2v.wv.word_vec("confessed")[0:50]

array([ -4.27130000e-01,   1.34660000e-01,  -3.49210000e-01,
         4.81920000e-01,  -4.93770000e-01,  -3.87940000e-02,
        -1.36500000e+00,  -6.52720000e-02,  -2.83830000e-01,
        -5.63370000e-01,  -2.60910000e-01,  -2.14390000e-02,
        -1.68270000e-01,  -1.90030000e-01,   1.46610000e-01,
         4.86780000e-01,   1.23060000e-01,  -3.40320000e-01,
         1.81250000e-01,   5.68720000e-01,  -1.14460000e-03,
        -5.96980000e-01,   1.30130000e-01,   7.11450000e-02,
        -5.22910000e-02,   1.61650000e-01,  -2.21380000e-01,
        -4.95220000e-01,   1.44850000e-01,  -1.23160000e-01,
         9.43030000e-02,   2.70700000e-01,   3.47260000e-02,
         4.33490000e-01,   8.71990000e-02,  -1.09690000e-01,
        -3.34760000e-01,   2.29290000e-01,   1.56970000e-01,
         1.58630000e-01,  -1.97850000e-01,   5.14440000e-02,
        -4.40730000e-01,   2.08360000e-01,   7.55970000e-02,
        -3.68940000e-01,  -1.75310000e-02,   5.62600000e-02,
         3.93250000e-01,

In [46]:
idx=word_to_ix['confessed']

In [47]:
weights[idx][0:50]

array([ -4.27130014e-01,   1.34660006e-01,  -3.49209994e-01,
         4.81920004e-01,  -4.93770003e-01,  -3.87939997e-02,
        -1.36500001e+00,  -6.52720034e-02,  -2.83829987e-01,
        -5.63369989e-01,  -2.60910004e-01,  -2.14389991e-02,
        -1.68270007e-01,  -1.90029994e-01,   1.46610007e-01,
         4.86779988e-01,   1.23060003e-01,  -3.40319991e-01,
         1.81250006e-01,   5.68719983e-01,  -1.14459998e-03,
        -5.96979976e-01,   1.30129993e-01,   7.11449981e-02,
        -5.22909984e-02,   1.61650002e-01,  -2.21379995e-01,
        -4.95220006e-01,   1.44850001e-01,  -1.23159997e-01,
         9.43029970e-02,   2.70700008e-01,   3.47260013e-02,
         4.33490008e-01,   8.71990025e-02,  -1.09690003e-01,
        -3.34760010e-01,   2.29289994e-01,   1.56969994e-01,
         1.58629999e-01,  -1.97850004e-01,   5.14440015e-02,
        -4.40730006e-01,   2.08360001e-01,   7.55970031e-02,
        -3.68939996e-01,  -1.75310001e-02,   5.62600009e-02,
         3.93249989e-01,

In [49]:
W2V_DIM = 300
HIDDEN_DIM = 80
NUM_LAYERS = 3
DROPOUT = 0.2

In [50]:
class GruClassifierW2vec(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, num_layers, vocab_size, label_size, pre_trained_weights, dropout):
        super(GruClassifierW2vec, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data=torch.Tensor(pre_trained_weights)
        self.gru = nn.GRU(input_size = embedding_dim,
                            hidden_size = hidden_dim,
                            num_layers = num_layers,
                            dropout = dropout)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # the first is the hidden h
        return (Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)).cuda())

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        x = embeds.view(len(sentence), 1, -1)
        for i in range(self.num_layers):
            gru_out, self.hidden = self.gru(x, self.hidden)
        y  = self.hidden2label(gru_out[-1])
        log_probs = F.log_softmax(y)
        return log_probs

In [51]:
model = GruClassifierW2vec(embedding_dim=W2V_DIM,
                            hidden_dim=HIDDEN_DIM,
                            num_layers=NUM_LAYERS,
                            vocab_size=VOCAB_SIZE,
                            label_size=NUM_LABELS,
                            pre_trained_weights = weights,
                            dropout = DROPOUT)

In [52]:
model.cuda()

GruClassifierW2vec(
  (word_embeddings): Embedding(28307, 300)
  (gru): GRU(300, 80, num_layers=3, dropout=0.2)
  (hidden2label): Linear(in_features=80, out_features=3)
)

In [53]:
avg=int((EAP_size+HPL_size+MWS_size)/3)
avg

6526

In [54]:
print(float(avg/EAP_size))
print(float(avg/HPL_size))
print(float(avg/MWS_size))

0.8260759493670886
1.1581188997338066
1.0797485109199205


In [55]:
EAP_size*0.826

6525.4

In [56]:
HPL_size*1.158

6525.33

In [57]:
MWS_size*1.079

6521.476

In [58]:
label_to_ix

{'EAP': 0, 'HPL': 1, 'MWS': 2}

In [59]:
## Loss function with mask to compensate class inbalance
mask=torch.cuda.FloatTensor((0.826,1.158,1.079))
#loss_function = nn.CrossEntropyLoss(weight=mask)

In [60]:
loss_function = nn.CrossEntropyLoss()
#print(loss_function.weight)
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [61]:
sample=train_data[2][0]
sample

'the lady whom was much astonished to hear addressed as madame joyeuse after the description of madame joyeuse she had just given blushed up to the eyebrows and seemed exceedingly abashed at the reproof '

In [62]:
sample_context=Variable(make_context_vector(sample,word_to_ix)).cuda()
sample_context

Variable containing:
 45
 53
 54
 16
 55
 56
 57
 58
 59
 60
 61
 62
 63
 45
 64
  8
 61
 62
 65
 10
 66
 67
 68
 69
 57
 45
 70
 14
 71
 72
 73
 74
 45
 75
[torch.cuda.LongTensor of size 34 (GPU 0)]

In [63]:
out=model(sample_context)
out

/home/ubuntu/Envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Variable containing:
-1.0662 -1.1038 -1.1268
[torch.cuda.FloatTensor of size 1x3 (GPU 0)]

In [66]:
batch_size = 30
n_iters = 3000
num_epochs = n_iters/(len(x_train) / batch_size)
num_epochs = int(num_epochs)
num_epochs

5

In [68]:
iter = 0
for epoch in range(num_epochs):
    for (sent,label) in train_data:
        # Step 1 - clear the gradients
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = model.init_hidden()
        
        ## Step 2- Prepare input and label
        context_vec = Variable(make_context_vector(sent, word_to_ix)).cuda()
        target = Variable(make_target(label, label_to_ix)).cuda()
        
        # Step 3 - Run forward pass
        output = model(context_vec)  
        
        # Step 4 - Compute loss, gradients, update parameters
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        
        iter+=1     
        ## Calculate final accuracy
        if iter % 1000 ==0:
            correct = 0
            total = 0
            for (sent,label) in valid_data:
                context_vec = Variable(make_context_vector(sent, word_to_ix)).cuda()
                target = Variable(make_target(label, label_to_ix)).cuda()
                output = model(context_vec)
                _,predicted = torch.max(output.data,1)
                total += target.size(0)
                correct += (predicted[0] == make_target(label, label_to_ix)).sum()
            accuracy = 100 * correct/total
            print('Iterations: {}. Loss: {}. Accuracy: {}'.format(iter,loss.data[0],accuracy))

/home/ubuntu/Envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Iterations: 1000. Loss: 0.0010656398953869939. Accuracy: 75.48518896833504
Iterations: 2000. Loss: 5.364274329622276e-05. Accuracy: 75.74055158324822
Iterations: 3000. Loss: 0.0001517419150331989. Accuracy: 75.79162410623084
Iterations: 4000. Loss: 0.0003995097358711064. Accuracy: 75.79162410623084
Iterations: 5000. Loss: 0.0010583758121356368. Accuracy: 75.25536261491318
Iterations: 6000. Loss: 0.00024256148026324809. Accuracy: 75.79162410623084
Iterations: 7000. Loss: 0.006435029674321413. Accuracy: 76.22574055158324
Iterations: 8000. Loss: 0.00018463814922142774. Accuracy: 73.56996935648621
Iterations: 9000. Loss: 2.706014311115723e-05. Accuracy: 72.88049029622063
Iterations: 10000. Loss: 3.421248038648628e-05. Accuracy: 73.1103166496425
Iterations: 11000. Loss: 0.00024136967840604484. Accuracy: 74.41266598569969
Iterations: 12000. Loss: 0.00011777184408856556. Accuracy: 74.1317671092952
Iterations: 13000. Loss: 0.06839083880186081. Accuracy: 73.54443309499489
Iterations: 14000. Los

In [69]:
## Making predictions on Test set

In [70]:
n=3
bow_vec = Variable(make_context_vector(valid_data[n][0], word_to_ix))
print("-"*20 + " INPUT "+"-"*30)
print("TRUE LABEL = {}".format(valid_data[n][1]))
print("SENTENCE = {}".format(valid_data[n][0]))
print("-"*20 + " PREDICTION "+"-"*30)
log_probs = model(bow_vec)
_,predicted = torch.max(log_probs.data,1)
print("PRED = {}".format(predicted[0]))
print("PRED = {}".format(list(label_to_ix.keys())[list(label_to_ix.values()).index(predicted[0])]))
##print("LOG_PROB = {}".format(log_probs))
print("PROBS = {}".format(F.softmax(log_probs)))

-------------------- INPUT ------------------------------
TRUE LABEL = EAP
SENTENCE = have indeed no abhorrence of danger except in its absolute effect in terror 
-------------------- PREDICTION ------------------------------
PRED = 0
PRED = EAP
PROBS = Variable containing:
 0.8685  0.1270  0.0045
[torch.cuda.FloatTensor of size 1x3 (GPU 0)]



/home/ubuntu/Envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ubuntu/Envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [71]:
from sklearn.metrics import log_loss

In [72]:
bow_vec = Variable(make_context_vector(valid_data[10][0], word_to_ix))

In [73]:
def calculate_log_loss(valid_data, model, label_to_ix, word_to_ix):
    true_label = np.zeros((len(valid_data),1))
    results_valid = np.zeros((len(valid_data),len(label_to_ix)))
    for i in range(len(valid_data)):
        bow_vec = Variable(make_context_vector(valid_data[i][0], word_to_ix))
        log_probs = model(bow_vec)
        pred = F.softmax(log_probs,dim=1).data.cpu().numpy()
        results_valid[i]=pred
        true_label[i]=label_to_ix[valid_data[i][1]]
    return log_loss(true_label,results_valid)

In [74]:
calculate_log_loss(valid_data, model, label_to_ix, word_to_ix)

/home/ubuntu/Envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


2.1709497185753852

In [ ]:
def make_preds(model,test):
    my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
    my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
    for i in range(len(test['phrase_preprocessed'])):
        sample=test['phrase_preprocessed'][i]
        #print(sample)
        sample_context=Variable(make_context_vector(sample,word_to_ix)).cuda()
        log_prob=model(sample_context)
        probs=F.softmax(log_prob)
        my_sub.loc[i] = [test['id'][i], probs.data[0][0],probs.data[0][1],probs.data[0][2]]
    return my_sub

In [ ]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

In [ ]:
my_sub

In [ ]:
preds=make_preds(model,test)

In [ ]:
preds.tail()

In [ ]:
preds.to_csv('roberto_new_12.csv',index=False)